Name : Parth Kapadia

In [ ]:
!pip install rake_nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install rapidfuzz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sparse_dot_topn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import rake_nltk as rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk.corpus
from tqdm import tqdm
nltk.download('stopwords')
from nltk.corpus import stopwords
from rapidfuzz import fuzz
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df_amazon = pd.read_excel("/content/excel_of_amz_com-ecommerce_sample.xlsx")
df_flipkart = pd.read_excel("/content/excel_of_flipkart_com-ecommerce_sample.xlsx")

In [ ]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   uniq_id                  20000 non-null  object
 1   crawl_timestamp          20000 non-null  object
 2   product_url              20000 non-null  object
 3   product_name             20000 non-null  object
 4   product_category_tree    20000 non-null  object
 5   pid                      20000 non-null  object
 6   retail_price             20000 non-null  int64 
 7   discounted_price         20000 non-null  int64 
 8   image                    19997 non-null  object
 9   is_FK_Advantage_product  20000 non-null  bool  
 10  description              19998 non-null  object
 11  product_rating           20000 non-null  object
 12  overall_rating           20000 non-null  object
 13  brand                    14136 non-null  object
 14  product_specifications   19986 non-nul

In [ ]:
df_flipkart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In [ ]:
print((df_amazon['product_rating'] == 'No rating available').sum())
print((df_amazon['overall_rating'] == 'No rating available').sum())
print((df_flipkart['product_rating'] == 'No rating available').sum())
print((df_flipkart['overall_rating'] == 'No rating available').sum())

18151
18151
18151
18151


In [ ]:
columns_to_drop = ['crawl_timestamp', 'product_url', 'image', 'product_rating', 'overall_rating', "product_specifications", "is_FK_Advantage_product", "product_category_tree", 'pid', 'uniq_id', 'brand']
df_amazon.drop(columns_to_drop, axis = 1, inplace = True)
df_flipkart.drop(columns_to_drop, axis = 1, inplace = True)

In [ ]:
df_amazon

,product_name,retail_price,discounted_price,description
0,Alisha Solid Women's Cycling Shorts,982,438,Key Features of Alisha Solid Women's Cycling S...
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW Bellies,991,551,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha Solid Women's Cycling Shorts,694,325,Key Features of Alisha Solid Women's Cycling S...
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Specifications of Sicons All Purpose Arnica Do...
...,...,...,...,...
19995,WALLDESIGN SMALL VINYL STICKER,1498,876,Buy WallDesign Small Vinyl Sticker for Rs.730 ...
19996,WALLMANTRA LARGE VINYL STICKERS STICKER,1415,1424,Buy Wallmantra Large Vinyl Stickers Sticker fo...
19997,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196,Buy Elite Collection Medium Acrylic Sticker fo...
19998,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364,Buy Elite Collection Medium Acrylic Sticker fo...


In [ ]:
df_flipkart

,product_name,retail_price,discounted_price,description
0,Alisha Solid Women's Cycling Shorts,999.0,379.0,Key Features of Alisha Solid Women's Cycling S...
1,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW Bellies,999.0,499.0,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha Solid Women's Cycling Shorts,699.0,267.0,Key Features of Alisha Solid Women's Cycling S...
4,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,Specifications of Sicons All Purpose Arnica Do...
...,...,...,...,...
19995,WallDesign Small Vinyl Sticker,1500.0,730.0,Buy WallDesign Small Vinyl Sticker for Rs.730 ...
19996,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0,Buy Wallmantra Large Vinyl Stickers Sticker fo...
19997,Elite Collection Medium Acrylic Sticker,1299.0,999.0,Buy Elite Collection Medium Acrylic Sticker fo...
19998,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,Buy Elite Collection Medium Acrylic Sticker fo...


In [ ]:
df_flipkart["description"].iloc[1]

"FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Price: Rs. 22,646 â€¢ Fine deep seating experience â€¢ Save Space with the all new click clack Sofa Bed â€¢ Easy to fold and vice versa with simple click clack mechanism â€¢ Chrome legs with mango wood frame for long term durability â€¢ Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience â€¢ A double bed that can easily sleep two,Specifications of FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contempora

In [ ]:
df_amazon.description = df_amazon.description.astype(str)
df_flipkart.description = df_flipkart.description.astype(str)

In [ ]:
def low_rem_uni(x):
  x = x.lower()
  x = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x)
  x = re.sub(' +', ' ', x)
  return x

stop_words = stopwords.words('english')

for name in ['product_name', 'description']:
  print(name)
  df_amazon[name] = df_amazon[name].apply(lambda x: low_rem_uni(x))
  df_flipkart[name] = df_flipkart[name].apply(lambda x: low_rem_uni(x))
  df_amazon[name] = df_amazon[name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
  df_flipkart[name] = df_flipkart[name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

product_name
description


In [ ]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_name      20000 non-null  object
 1   retail_price      20000 non-null  int64 
 2   discounted_price  20000 non-null  int64 
 3   description       20000 non-null  object
dtypes: int64(2), object(2)
memory usage: 625.1+ KB


In [ ]:
df_flipkart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_name      20000 non-null  object 
 1   retail_price      19922 non-null  float64
 2   discounted_price  19922 non-null  float64
 3   description       20000 non-null  object 
dtypes: float64(2), object(2)
memory usage: 625.1+ KB


In [ ]:
df_flipkart["description"].iloc[1]

'fabhomedecor fabric double sofa bed finish color leatherette black mechanism type pull price rs 22646 fine deep seating experience save space new click clack sofa bed easy fold vice versa simple click clack mechanism chrome legs mango wood frame long term durability double cushioned sofa bed provide extra softness make fine seating experience double bed easily sleep twospecifications fabhomedecor fabric double sofa bed finish color leatherette black mechanism type pull installation demo installation demo details installation demo product done free cost part purchase service partner visit location within 72 business hours delivery product box 1 sofa bed general brand fabhomedecor mattress included delivery condition knock storage included mechanism type pull type sofa bed style contemporary modern filling material microfiber seating capacity 3 seater upholstery type na upholstery included bed size double shape square suitable living room model number fhd112 care instructions avoid outd

In [ ]:
df_flipkart

,product_name,retail_price,discounted_price,description
0,alisha solid womens cycling shorts,999.0,379.0,key features alisha solid womens cycling short...
1,fabhomedecor fabric double sofa bed,32157.0,22646.0,fabhomedecor fabric double sofa bed finish col...
2,aw bellies,999.0,499.0,key features aw bellies sandals wedges heel ca...
3,alisha solid womens cycling shorts,699.0,267.0,key features alisha solid womens cycling short...
4,sicons purpose arnica dog shampoo,220.0,210.0,specifications sicons purpose arnica dog shamp...
...,...,...,...,...
19995,walldesign small vinyl sticker,1500.0,730.0,buy walldesign small vinyl sticker rs730 onlin...
19996,wallmantra large vinyl stickers sticker,1429.0,1143.0,buy wallmantra large vinyl stickers sticker rs...
19997,elite collection medium acrylic sticker,1299.0,999.0,buy elite collection medium acrylic sticker rs...
19998,elite collection medium acrylic sticker,1499.0,1199.0,buy elite collection medium acrylic sticker rs...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')
flipkart_tf_idf_matrix = tfidf_vectorizer.fit_transform(df_flipkart['description'])
amazon_tf_idf_matrix = tfidf_vectorizer.transform(df_flipkart['description'])

In [ ]:
flipkart_tf_idf_matrix[0][0]

<1x28272 sparse matrix of type '<class 'numpy.float64'>'
	with 74 stored elements in Compressed Sparse Row format>

In [ ]:
amazon_tf_idf_matrix[0][0]

<1x28272 sparse matrix of type '<class 'numpy.float64'>'
	with 74 stored elements in Compressed Sparse Row format>

In [ ]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))


import time
t1 = time.time()

# adjust lower bound: 0.8
# keep top 10 similar results
matches = awesome_cossim_top(flipkart_tf_idf_matrix, amazon_tf_idf_matrix.transpose(), 3)

t = time.time()-t1
print("finished in:", t)

finished in: 8.524242639541626


In [ ]:
c = matches.tocoo()                                                                                        
df = pd.DataFrame({'flipkart_index': c.row, 'amazon_index': c.col, 'similarity score': c.data})                                   
df

,flipkart_index,amazon_index,similarity score
0,0,0,1.000000
1,0,15,0.982487
2,0,13,0.981669
3,1,1,1.000000
4,1,16,0.976078
...,...,...,...
59989,19998,19945,1.000000
59990,19998,19998,1.000000
59991,19999,19993,1.000000
59992,19999,19999,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59994 entries, 0 to 59993
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   flipkart_index    59994 non-null  int32  
 1   amazon_index      59994 non-null  int32  
 2   similarity score  59994 non-null  float64
dtypes: float64(1), int32(2)
memory usage: 937.5 KB


In [ ]:
df["retail_price_ratio"] = np.where(pd.isna(df_flipkart['retail_price'].iloc[df["flipkart_index"]]), 0.5, 1)

for index, row in df.iterrows():
  if(row['retail_price_ratio'] == 1):
    if(df_flipkart['retail_price'].iloc[int(row["flipkart_index"])]<df_amazon['retail_price'].iloc[int(row["amazon_index"])]):
      df['retail_price_ratio'].iloc[index] = df_flipkart['retail_price'].iloc[int(row["flipkart_index"])]/df_amazon['retail_price'].iloc[int(row["amazon_index"])]
    else:
      df['retail_price_ratio'].iloc[index] = df_amazon['retail_price'].iloc[int(row["amazon_index"])]/df_flipkart['retail_price'].iloc[int(row["flipkart_index"])]

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df["discounted_price_ratio"] = np.where(pd.isna(df_flipkart['retail_price'].iloc[df["flipkart_index"]]), 0.5, 1)

for index, row in df.iterrows():
  if(row['discounted_price_ratio'] == 1):
    if(df_flipkart['discounted_price'].iloc[int(row["flipkart_index"])]<df_amazon['discounted_price'].iloc[int(row["amazon_index"])]):
      df['discounted_price_ratio'].iloc[index] = df_flipkart['discounted_price'].iloc[int(row["flipkart_index"])]/df_amazon['discounted_price'].iloc[int(row["amazon_index"])]
    else:
      df['discounted_price_ratio'].iloc[index] = df_amazon['discounted_price'].iloc[int(row["amazon_index"])]/df_flipkart['discounted_price'].iloc[int(row["flipkart_index"])]

In [ ]:
df

,flipkart_index,amazon_index,similarity score,retail_price_ratio,discounted_price_ratio
0,0,0,1.000000,0.982983,0.865297
1,0,15,0.982487,0.995996,0.873272
2,0,13,0.981669,0.990991,0.791232
3,1,1,1.000000,0.999565,0.777652
4,1,16,0.976078,0.999782,0.790436
...,...,...,...,...,...
59989,19998,19945,1.000000,0.992662,0.900826
59990,19998,19998,1.000000,0.995330,0.879032
59991,19999,19993,1.000000,0.855237,0.858247
59992,19999,19999,1.000000,0.989993,0.801123


In [ ]:
df['product_name_similarity'] = 0

for index, rows in tqdm(df.iterrows()):
  df['product_name_similarity'] = (fuzz.ratio(df_flipkart['product_name'].iloc[int(row['flipkart_index'])],
                                             df_amazon['product_name'].iloc[int(row['amazon_index'])]))/100

59994it [00:19, 3115.94it/s]


In [ ]:
df["final_score"] = df['similarity score']*df['retail_price_ratio']*df['discounted_price_ratio']*df['product_name_similarity']

In [ ]:
df

,flipkart_index,amazon_index,similarity score,retail_price_ratio,discounted_price_ratio,product_name_similarity,final_score
0,0,0,1.000000,0.982983,0.865297,1.0,0.850572
1,0,15,0.982487,0.995996,0.873272,1.0,0.854543
2,0,13,0.981669,0.990991,0.791232,1.0,0.769730
3,1,1,1.000000,0.999565,0.777652,1.0,0.777313
4,1,16,0.976078,0.999782,0.790436,1.0,0.771360
...,...,...,...,...,...,...,...
59989,19998,19945,1.000000,0.992662,0.900826,1.0,0.894216
59990,19998,19998,1.000000,0.995330,0.879032,1.0,0.874927
59991,19999,19993,1.000000,0.855237,0.858247,1.0,0.734005
59992,19999,19999,1.000000,0.989993,0.801123,1.0,0.793106


In [ ]:
final_cols = ['flipkart_index', 'amazon_index', 'final_score']

df_final = df[final_cols]

In [ ]:
df_final

,flipkart_index,amazon_index,final_score
0,0,0,0.850572
1,0,15,0.854543
2,0,13,0.769730
3,1,1,0.777313
4,1,16,0.771360
...,...,...,...
59989,19998,19945,0.894216
59990,19998,19998,0.874927
59991,19999,19993,0.734005
59992,19999,19999,0.793106


In [ ]:
df_final['flipkart_index'].nunique()

19998

In [ ]:
merger = pd.DataFrame(df_final.groupby(['flipkart_index'])['final_score'].max()).reset_index().drop_duplicates()
merger

,flipkart_index,final_score
0,0,0.854543
1,1,0.777313
2,2,0.898374
3,3,0.815662
4,4,0.769556
...,...,...
19993,19995,0.886258
19994,19996,0.888803
19995,19997,0.847016
19996,19998,0.894216


In [ ]:
df_new = pd.merge(df_final, merger, how = 'right', on = ['flipkart_index', 'final_score'])

In [ ]:
vals = df_new['flipkart_index'].value_counts()[:14].to_dict()

rem_ind = []

for key, val in vals.items():
  temp = df_new[df_new['flipkart_index'] == key]
  ctr = 0
  for ind, row in temp.iterrows():
    if(ctr == val-1):
      pass
    else:
      rem_ind.append(ind)
    ctr+=1

In [ ]:
df_new.drop(df_new.index[rem_ind], inplace = True)
df_new = df_new.reset_index()

In [ ]:
df_flip_join = df_flipkart[['product_name', 'retail_price', 'discounted_price']]
df_amaz_join = df_amazon[['product_name', 'retail_price', 'discounted_price']]

df_new["Flipkart product name"] = " "
df_new['Flipkart retail price'] = 0
df_new['Flipkart discouonted price'] = 0

df_new["Amazon product name"] = " "
df_new['Amazon retail price'] = 0
df_new['Amazon discouonted price'] = 0

for index, row in df_new.iterrows():
  df_new["Flipkart product name"] = df_flipkart['product_name'].iloc[int(row['flipkart_index'])]
  df_new['Flipkart retail price'] = df_flipkart['retail_price'].iloc[int(row['flipkart_index'])]
  df_new['Flipkart discouonted price'] = df_flipkart['discounted_price'].iloc[int(row['flipkart_index'])]

  df_new["Amazon product name"] = df_amazon['product_name'].iloc[int(row['amazon_index'])]
  df_new['Amazon retail price'] = df_amazon['retail_price'].iloc[int(row['amazon_index'])]
  df_new['Amazon discouonted price'] = df_amazon['discounted_price'].iloc[int(row['amazon_index'])]
    

In [ ]:
df_new.drop(['index', 'flipkart_index', 'amazon_index', 'final_score'], axis = 1, inplace = True)

**We've obtained the final table**

In [ ]:
df_new

,Flipkart product name,Flipkart retail price,Flipkart discouonted price,Amazon product name,Amazon retail price,Amazon discouonted price
0,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
1,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
2,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
3,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
4,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
...,...,...,...,...,...,...
19993,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
19994,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
19995,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
19996,elite collection medium acrylic sticker,1499.0,999.0,elite collection medium acrylic sticker,1484,1247
